In [1]:
from huggingface_hub import login

login("")

In [2]:
# !pip install --upgrade pip  # enable PEP 660 support
# !cd LLaVA && pip install -e .
!pip install transformers accelerate torchvision

In [4]:
# from llava.model.builder import load_pretrained_model
# from llava.mm_utils import get_model_name_from_path
# from llava.eval.run_llava import eval_model

# model_path = "liuhaotian/llava-v1.5-7b"

# tokenizer, model, image_processor, context_len = load_pretrained_model(
#     model_path=model_path,
#     model_base=None,
#     model_name=get_model_name_from_path(model_path)
# )
import transformers
import torch
from PIL import Image
import os
import json
from tqdm import tqdm
from transformers import Blip2Processor, Blip2ForConditionalGeneration
from PIL import Image


processor = Blip2Processor.from_pretrained("Salesforce/blip2-flan-t5-xl")
model = Blip2ForConditionalGeneration.from_pretrained("Salesforce/blip2-flan-t5-xl")

device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

model-00002-of-00002.safetensors:  29%|##8       | 1.68G/5.81G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/168 [00:00<?, ?B/s]

Blip2ForConditionalGeneration(
  (vision_model): Blip2VisionModel(
    (embeddings): Blip2VisionEmbeddings(
      (patch_embedding): Conv2d(3, 1408, kernel_size=(14, 14), stride=(14, 14))
    )
    (encoder): Blip2Encoder(
      (layers): ModuleList(
        (0-38): 39 x Blip2EncoderLayer(
          (self_attn): Blip2Attention(
            (dropout): Dropout(p=0.0, inplace=False)
            (qkv): Linear(in_features=1408, out_features=4224, bias=True)
            (projection): Linear(in_features=1408, out_features=1408, bias=True)
          )
          (layer_norm1): LayerNorm((1408,), eps=1e-06, elementwise_affine=True)
          (mlp): Blip2MLP(
            (activation_fn): GELUActivation()
            (fc1): Linear(in_features=1408, out_features=6144, bias=True)
            (fc2): Linear(in_features=6144, out_features=1408, bias=True)
          )
          (layer_norm2): LayerNorm((1408,), eps=1e-06, elementwise_affine=True)
        )
      )
    )
    (post_layernorm): LayerNorm((

In [5]:
#connect with google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
!unzip /content/drive/MyDrive/place_centric_data.zip

流式输出内容被截断，只能显示最后 5000 行内容。
  inflating: place_centric_data/place_centric_images/ChIJ87PpjrHjDDkRyRazVZzAhbo.jpg  
  inflating: place_centric_data/place_centric_images/ChIJ87v_k9IEdkgR2vAfuejGjpA.jpg  
  inflating: place_centric_data/place_centric_images/ChIJ88B6nbJC1moRSK9UOBAx0nQ.jpg  
  inflating: place_centric_data/place_centric_images/ChIJ890DDNQEdkgRH34fyENRFuo.jpg  
  inflating: place_centric_data/place_centric_images/ChIJ89rP0LRC1moRjJ9h1rkpnpk.jpg  
  inflating: place_centric_data/place_centric_images/ChIJ89rP2yNd1moRhxavtiiiP8o.jpg  
  inflating: place_centric_data/place_centric_images/ChIJ8aCTskzBhkcRyKfjydozEV0.jpg  
  inflating: place_centric_data/place_centric_images/ChIJ8ajsoAT21moR6DqI6mqVO5c.jpg  
  inflating: place_centric_data/place_centric_images/ChIJ8aMop13iDDkR6SXfjDFdyvI.jpg  
  inflating: place_centric_data/place_centric_images/ChIJ8aPAF8Vn1moRWfE5dzq3S2g.jpg  
  inflating: place_centric_data/place_centric_images/ChIJ8ar7EspC1moRnU4TGkmE_n8.jpg  
  inflating: pla

In [7]:
import torch
import json
from tqdm import tqdm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [8]:
from PIL import Image
from transformers import TextStreamer

def predict(image: Image, prompt: str) -> str:
    """
    image: PIL.Image
    question: str
    """

    inputs = processor(images=[image], text=[prompt], return_tensors="pt").to(device, torch.float16)

    # 模型推理
    with torch.no_grad():
        output_ids = model.generate(**inputs, max_new_tokens=512, do_sample=True,temperature=0.2)
    raw_answer = processor.batch_decode(output_ids, skip_special_tokens=True)[0].strip()

    answer = raw_answer.strip().upper()
    if "ASSISTANT:" in answer:
      answer = raw_answer.split("ASSISTANT:")[1].strip()
      answer = answer.split()[0]
      answer = answer.rstrip(':')
    else:
      answer = raw_answer.strip()

    # with torch.inference_mode():
    #   # mute the model
    #     output_ids = model.generate(
    #         input_ids,
    #         images=image_tensor,
    #         do_sample=True,
    #         temperature=0.2,
    #         max_new_tokens=512,
    #         streamer=streamer,
    #         use_cache=True,
    #         stopping_criteria=[stopping_criteria])

    # outputs = tokenizer.decode(output_ids[0]).strip()
    # conv.messages[-1][-1] = outputs

    return answer

In [9]:
def get_question_prompt(question):

  intention_driven_qa_template = """{question} Here are the options:
A. {answer_a}
B. {answer_b}
C. {answer_c}
D. {answer_d}

Reasoning: {reasoning}"""

  return intention_driven_qa_template.format(
    question=question["question"],
    answer_a=question["answer_a"],
    answer_b=question["answer_b"],
    answer_c=question["answer_c"],
    answer_d=question["answer_d"],
    tags_a=question["a_tags"][0],
    tags_b=question["b_tags"][0],
    tags_c=question["c_tags"][0],
    tags_d=question["d_tags"][0],
    attributes=", ".join(question["attributes"]),
    reasoning=question["reasoning"]
  )

def parse_model_answer(answer, model_answer):
  if model_answer == '':
      return False

  # Step 1: match prediction
  match_candidate_list = [
      f"{answer}", f"{answer}.", f"{answer})", f"{answer},", f"{answer})."
  ]
  if model_answer in match_candidate_list:
      return True

  # directly match the first word
  if model_answer[0].lower() == answer.lower():
      return True

  return False

In [ ]:
qa_pairs = json.load(open("/content/drive/MyDrive/qa_pairs_tagged.json"))
# qa_pairs = json.load(open("/content/drive/MyDrive/qa_pairs.json"))
correct = 0
answers_raw = {}


########################################
# Part 1: Output the reasoning steps
########################################
for key in tqdm(list(qa_pairs.keys())):
  question = qa_pairs[key]
  prompt_q = question["question"]
  image = Image.open(f"./place_centric_data/place_centric_images/{key}.jpg")

  img_name = key
  gt_attr = qa_pairs[img_name]["attributes"]
  gt_answer = qa_pairs[img_name]["answer"]
  qa_info = qa_pairs.get(img_name, {})

  options = {key[-1].upper(): value for key, value in qa_info.items() if key.startswith("answer_")}
  options_text = "\n".join([f"{key}: {value}" for key, value in options.items()])



    # 加载本地图片
  try:
      image = image.convert("RGB")
  except Exception as e:
      print(f"Error loading image {img_name}: {e}")
      answers_raw[img_name] = {
          "answer": "",
          "ground_truth": gt_attr,
          "correctness": "unknown"
      }
      continue



  prompt = f"""
  You are a Visual Instructor, an expert in visual reasoning and critical thinking, tasked with analyzing the provided image and developing a clear, step-by-step reasoning chain.

  Analyze the image and question thoroughly.
  Output your reasoning **step by step** in clear, logical sentences, separated by newlines.
  Do not output the final answer yet; focus only on the reasoning process

  Question: {prompt_q}
  Here are the options:
  {options_text}

  """

  answer = predict(image, prompt)

  # extract from <s> A,C,D</s>
  answer = answer.replace("<s>", "").replace("</s>", "").strip()

  question["reasoning"] = answer
  answers_raw[key] = {
      "reasoning": answer
  }


########################################
# Part 2: Single Pass
########################################
  prompt = get_question_prompt(question)

  prompt = """You are a Visual Instructor, continuing your analysis of the problem.
You now have the step-by-step reasoning chain from your earlier analysis.

Based on the reasoning steps you developed, determine the correct answer to the question.
Output **only** the letter of the correct option (A, B, C, or D).
Do not include any explanations, steps, or additional text."""+prompt
  answer = predict(image, prompt)

  # extract from <s> A,C,D</s>
  answer = answer.replace("<s>", "").replace("</s>", "").strip()

  answers_raw[key]["truth"] = question["answer"]
  answers_raw[key]["answer"] = answer
  answers_raw[key]["is_correct"] = parse_model_answer(question["answer"], answer)

  if answers_raw[key]["is_correct"]:
      correct += 1
########################################
# End Of Single Pass
########################################

# qa_pairs is a dict
print(correct)
print(correct / len(qa_pairs))


100%|██████████| 5828/5828 [5:19:07<00:00,  3.29s/it]

4949
0.8491763898421414


In [ ]:
# save answers_raw as json to the drive
with open('/content/drive/MyDrive/answers_step_eval_blip2.json', 'w') as f:
    json.dump(answers_raw, f)

In [ ]:
print(correct / len(qa_pairs))

0.0008579272477693892


In [ ]:
# blip2 reasoning 2 stage circular
from operator import truth
qa_reasoning = json.load(open('/content/drive/MyDrive/answers_step_eval_blip2.json'))
qa_pairs = json.load(open("/content/drive/MyDrive/qa_pairs_tagged.json"))
correct = 0
answers_raw = {}

########################################
# Circular Pass
########################################
for key in tqdm(list(qa_pairs.keys())):
  image = Image.open(f"./place_centric_data/place_centric_images/{key}.jpg")

  question_rotated = qa_pairs[key]
  question_rotated["reasoning"] = qa_reasoning[key]["reasoning"]

  is_correct = True
  answers = []
  truths = []

  for _ in range(4):
    question_rotated["answer_a"], question_rotated["answer_b"], question_rotated["answer_c"], question_rotated["answer_d"] = question_rotated["answer_b"], question_rotated["answer_c"], question_rotated["answer_d"], question_rotated["answer_a"]

    question_rotated["answer"] = {
        "B": "A",
        "C": "B",
        "D": "C",
        "A": "D"
    }[question_rotated["answer"]]
    truths.append(question_rotated["answer"])

    prompt = get_question_prompt(question_rotated)
    prompt = """You are a Visual Instructor, continuing your analysis of the problem.
You now have the step-by-step reasoning chain from your earlier analysis.

Based on the reasoning steps you developed, determine the correct answer to the question.
Output **only** the letter of the correct option (A, B, C, or D).
Do not include any explanations, steps, or additional text."""+prompt

    answer = predict(image, prompt)

    # extract from <s> A,C,D</s>
    answer = answer.replace("<s>", "").replace("</s>", "").strip()
    answers.append(answer)


    if not parse_model_answer(question_rotated["answer"], answer):
      is_correct = False
      break

  answers_raw[key] = {
      "truths": truths,
      "answers_raw": answers,
      "is_correct": is_correct
  }
  correct += answers_raw[key]["is_correct"]
########################################
# End Of Circular Pass
########################################

In [ ]:
# save answers_raw as json to the drive
with open('/content/drive/MyDrive/answers_step_eval_blip2_circular.json', 'w') as f:
    json.dump(answers_raw, f)
print(correct / len(qa_pairs))

0.5054907343857241


In [ ]:
# blip2 implicit reasoing circular
from operator import truth
qa_reasoning = json.load(open('/content/drive/MyDrive/answers_step_eval_blip2.json'))
qa_pairs = json.load(open("/content/drive/MyDrive/qa_pairs_tagged.json"))
correct = 0
answers_raw = {}

########################################
# Circular Pass
########################################
for key in tqdm(list(qa_pairs.keys())):
  image = Image.open(f"./place_centric_data/place_centric_images/{key}.jpg")

  question_rotated = qa_pairs[key]
  question_rotated["reasoning"] = qa_reasoning[key]["reasoning"]

  is_correct = True
  answers = []
  truths = []

  for _ in range(4):
    question_rotated["answer_a"], question_rotated["answer_b"], question_rotated["answer_c"], question_rotated["answer_d"] = question_rotated["answer_b"], question_rotated["answer_c"], question_rotated["answer_d"], question_rotated["answer_a"]

    question_rotated["answer"] = {
        "B": "A",
        "C": "B",
        "D": "C",
        "A": "D"
    }[question_rotated["answer"]]
    truths.append(question_rotated["answer"])



    prompt = get_question_prompt(question_rotated)
    prompt = """You are the Visual Instructor, an expert in visual reasoning and critical thinking.
    Your role is to guide your students in accurately interpreting visual information and answering related questions.
    Analyze the image and choose the correct option."""+prompt+"""
    Output Format:
    - First, respond with the letter of the option (A, B, C, or D).
    - Next, provide a step-by-step reasoning chain to justify your choice.
    """

    answer = predict(image, prompt)

    # extract from <s> A,C,D</s>
    answer = answer.replace("<s>", "").replace("</s>", "").strip()
    answers.append(answer)


    # if not parse_model_answer(question_rotated["answer"], answer):
    #   is_correct = False
    #   break

  answers_raw[key] = {
      "truths": truths,
      "answers_raw": answers,
      "is_correct": is_correct
  }
  correct += answers_raw[key]["is_correct"]
########################################
# End Of Circular Pass
########################################

  4%|▍         | 232/5828 [09:15<2:34:43,  1.66s/it]

In [ ]:
# save answers_raw as json to the drive
with open('/content/drive/MyDrive/answers_implicit_eval_blip2_single.json', 'w') as f:
    json.dump(answers_raw, f)
print(correct / len(qa_pairs))
# requires LLM assist